In [1]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
def load_images(folder):
        image_list=[]
        for filename in os.listdir(folder):
            img=cv2.imread(os.path.join(folder,filename))
            if img is not None:
                image_list.append(img)
        return image_list

In [16]:
tk.Tk().withdraw()
path=tkFileDialog.askdirectory()
print path


/home/edward/Project /Untitled Folder


In [54]:
image=load_images(path)

In [55]:
data=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")

In [56]:
data['melanoma'].head()

0    0
1    0
2    1
3    0
4    1
Name: melanoma, dtype: int64

In [57]:
image=np.array(image)
image.shape


(1, 64, 64, 3)

In [52]:
image=np.reshape(image,-1)

In [63]:
image=tf.reshape(image,[12288,1])

In [ ]:
image.shape

In [60]:
learning_rate=0.001
training_epoch=15
batch_size=100
display_step=1

n_hidden_1=256
n_hidden_2=256
n_hidden_3=256
n_hidden_4=256
n_input=12288
n_classes=2

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])

In [61]:
def multiplayer_perceptron(x,weights,biases):
    # first layer
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.relu(layer_1)
    # second layer
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.relu(layer_2)
    # third layer
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.relu(layer_3)
    # forth layer
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.relu(layer_4)
    # output layer
    out_layer=tf.matmul(layer_4,weights['out'])+biases['out']
    return out_layer

In [62]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.random_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_hidden_4,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'b3':tf.Variable(tf.random_normal([n_hidden_3])),
    'b4':tf.Variable(tf.random_normal([n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron(x,weights,biases)

# define loss and optimizer
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [ ]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training 
    for epoch in range(training_epoch):
        avg_cost=0.
        total_batch=batch_size
        for i in range(total_batch):
            batch_x=image
            batch_y=data['melanoma']
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            avg_cost+=c/total_batch
        if epoch%display_step==0:
               print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"
      # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: batch_x, y: batch_y})
        